In [1]:
from pyspark.sql import SparkSession
# Initialize Spark
spark = SparkSession.builder.appName("gpt_testing").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).

25/10/27 12:09:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is not supported
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:277)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
	at py4j.reflection.MethodInvoker.in

In [ ]:
from pyspark.sql import functions as F, Window as W

orders = spark.read.table("bronze.orders_raw")
ship = spark.read.table("bronze.shipment_events")
cust = spark.read.table("silver.customers_dim")

# Narrow to shipping-related events
ship_f = ship.filter(F.col("event_type").isin("PICKED", "PACKED", "SHIPPED"))

# Derive first shipped timestamp per order
w = W.partitionBy("order_id").orderBy(F.col("event_ts").asc())
ship_ranked = (
    ship_f
    .withColumn("rn", F.row_number().over(w))
    .filter((F.col("event_type")=="SHIPPED") & (F.col("rn")==1))
    .select("order_id", F.col("event_ts").alias("first_shipped_ts"))
)

# Join to orders (note: engineers recently added this cast for “consistency”)
orders_c = orders.withColumn("order_id_long", F.col("order_id").cast("long"))
ship_c   = ship_ranked.withColumn("order_id_long", F.col("order_id").cast("long"))

# Hint to improve join speed
joined = (
    orders_c.hint("broadcast")
    .join(ship_c.hint("broadcast"), on="order_id_long", how="left")
    .drop("order_id")  # keep long key
    .withColumnRenamed("order_id_long", "order_id")
)

# Attach latest customer data
full = (
    joined.join(cust, on="customer_id", how="left")
)

daily = (
    full
    .withColumn("ship_latency_hours", F.when(F.col("first_shipped_ts").isNotNull(),
                                             F.round(F.col("first_shipped_ts").cast("long") - F.col("order_ts").cast("long"))/3600))
    .withColumn("ds", F.to_date(F.coalesce("first_shipped_ts", "order_ts")))
    .groupBy("country", "ds")
    .agg(
        F.countDistinct(F.when(F.col("first_shipped_ts").isNotNull(), F.col("order_id"))).alias("shipped_orders"),
        F.avg("ship_latency_hours").alias("avg_ship_latency_hours"),
        F.sum(F.when(F.col("ship_latency_hours") > 48, F.lit(1)).otherwise(F.lit(0))).alias("late_shipments")
    )
)

(daily
 .repartition(200, "ds")
 .write
 .format("delta")
 .mode("overwrite")
 .partitionBy("ds")
 .saveAsTable("gold.daily_fulfillment_summary"))